### Intern take-home Assessment:

In [115]:
import os
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

* The question is:
"predict what profits MIGHT have been at each of the three candidate locations IF 
the locations had been open and operating during appropriate time periods 
from January 1st through October 30th of 2022. 
Based on these predictions, which location would have likely been the most profitable 
in the first 10 months of 2022 if it had been open for business?

This actually sounds more like hypothetical causal predictions regarding the profits, with Locations as the "intervention/Treatment/Exposure" variable.

### Data preprocessing:

In [3]:
#import all dataset with different format:
directory = "data"
dta_transac = []
dta_other = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        data_type = f.split('.')[0].split('_')[0].split('/')[1] #i.e. transactions, weather, holiday, location
        if data_type == 'transactions' and f.split('.')[-1] == 'txt':
            dta = pd.read_csv(f, sep = "\t")
            dta_transac.append(dta)
        elif data_type == 'transactions' and f.split('.')[-1] == 'json':
            dta = pd.read_json(f)
            dta_transac.append(dta)
        elif data_type == 'transactions' and f.split('.')[-1] == 'csv':
            dta = pd.read_csv(f)
            dta_transac.append(dta)
        else:
            dta = pd.read_csv(f)
            dta_other.append(dta)

In [4]:
#concatenate all of the transaction data together
data_transac = pd.concat(dta_transac)

In [5]:
#unify the format of the profit data using regular expression.
profit = []
for i in data_transac.profit:#.str[2:]:
    #print(i)
    #print(type(str(i)))
    res = re.sub('[-$]', '', str(i))
    profit.append(float(str(res)))
data_transac['profit'] = profit

##### Concatenate all other variables except transactions:

In [6]:
#firstly, combine the holiday with the location data:
dta = dta_other[0].merge(dta_other[1], how = 'left', on = 'date')
dta = dta.fillna(0) #fill out the holiday indicator to be 0 for missing values in holiday
dta.holiday = dta.holiday.astype(int)

In [7]:
#secondly, combine the merged data with location data (i.e., population, elevation)
dta = dta.merge(dta_other[2], how = 'left', on = 'location_id')

In [56]:
#thirdly, aggregate the profits for each location for each date:
dta_tran_agg = (data_transac
    .groupby(['location_id', 'date'])
    ['profit'].sum()
    .reset_index(level=[0,1]))
dta_tran_agg['date'] = pd.to_datetime(dta_tran_agg.date).dt.strftime('%m-%d-%Y')

In [57]:
dta_tran_agg

,location_id,date,profit
0,1,01-02-2019,430.96
1,1,01-03-2019,416.90
2,1,01-04-2019,280.16
3,1,01-05-2019,415.98
4,1,01-06-2019,412.20
...,...,...,...
7637,10,09-07-2022,432.25
7638,10,09-08-2022,405.04
7639,10,09-09-2022,124.59
7640,10,09-10-2022,374.87


In [91]:
dta

,location_id,date,temperature,pressure,humidity,cloudy,precipitation,holiday,population,elevation
0,3,2020-01-22,18.14,1035.058685,0.44,True,False,0,74138,505
1,8,2022-01-29,14.36,1027.253521,0.95,True,False,0,52185,398
2,5,2021-11-28,35.42,994.694836,0.37,False,False,0,12686,386
3,6,2021-10-12,37.94,1003.838028,0.11,True,True,0,86372,435
4,2,2020-12-03,23.36,1027.476526,0.60,False,False,0,32926,274
...,...,...,...,...,...,...,...,...,...,...
17273,6,2022-07-03,78.98,966.819249,0.88,True,True,0,86372,435
17274,6,2020-02-20,9.68,971.948357,0.53,False,False,0,86372,435
17275,10,2020-10-02,41.18,1013.873239,0.90,False,False,0,425336,266
17276,1,2019-12-15,15.08,990.457746,0.74,False,False,0,18428,375


In [70]:
#change the format to make sure that the transaction data and other information are using the same date format
dta['date'] = pd.to_datetime(dta.date).dt.strftime('%Y-%m-%d')
dta_tran_agg['date'] = pd.to_datetime(dta_tran_agg.date).dt.strftime('%Y-%m-%d')

In [73]:
#merge the aggregated transaction data with the other characteristics of the different location
final_data = dta_tran_agg.merge(dta, on = ['location_id', 'date'], how = 'left')
#use last observation carry forward for the missing data:
final_data = final_data.sort_values(['location_id', 'date'], ascending = [True, True])
final_data = final_data.ffill(axis = 0)

### Modeling

In [81]:
#define X and y data:
X = np.array(final_data.iloc[:, 3:])
y = np.array(final_data.iloc[:, 2])

In [84]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [123]:
from sklearn import metrics
metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [125]:
#split the data to training and testing:
#X_train, X_test, y_train, y_test = train_test_split()
lasso = Lasso(random_state=0, max_iter=10000)
alphas = np.logspace(-4, -0.5, 30)
tuned_parameters = [{"alpha": alphas}]
n_folds = 5
clf = GridSearchCV(lasso, tuned_parameters, cv=n_folds, refit=True, scoring='neg_mean_squared_error')
clf.fit(X, y)
best_params = clf.best_params_
model = clf.best_estimator_
score = clf.best_score_

In [120]:
clf.cv_results_

{'mean_fit_time': array([0.0120542 , 0.02371349, 0.0098978 , 0.0345118 , 0.02376399,
        0.01623611, 0.02529807, 0.02916112, 0.03901191, 0.02121887,
        0.01761088, 0.03195624, 0.03515086, 0.02836967, 0.02421446,
        0.03196068, 0.02499423, 0.05088301, 0.02816358, 0.02321358,
        0.02775588, 0.02492247, 0.0133472 , 0.01749611, 0.01918931,
        0.02450719, 0.02474127, 0.02141933, 0.01858087, 0.02311859]),
 'std_fit_time': array([0.00532864, 0.01032326, 0.00517018, 0.01487862, 0.00720043,
        0.00875989, 0.00757787, 0.01269826, 0.02990123, 0.0116807 ,
        0.01330567, 0.01914588, 0.01248997, 0.00872333, 0.01178182,
        0.00950611, 0.01312294, 0.01791085, 0.02190876, 0.00956946,
        0.01355197, 0.01604505, 0.00546621, 0.00437622, 0.00235939,
        0.01149665, 0.01454114, 0.01467519, 0.00742774, 0.01553463]),
 'mean_score_time': array([0.00589218, 0.00605931, 0.00493312, 0.02569232, 0.01832519,
        0.01820898, 0.0156899 , 0.02952089, 0.01344023, 0.00

In [92]:
#retrieve the data to be predicted from location 11, 12, 13:
data_new = dta.loc[dta.location_id.isin([11, 12, 13])]

In [95]:
X_new = scaler.transform(np.array(data_new.iloc[:, 2:]))

In [96]:
X_new

array([[ 1.27535861, -0.56733972,  0.91063665, ...,  0.        ,
        -0.03786037, -0.48899485],
       [ 0.58880963,  0.45330921,  0.97630605, ...,  0.        ,
        -0.03786037, -0.48899485],
       [-0.48907227, -1.41048449, -1.4862966 , ...,  0.        ,
        -0.03786037, -0.48899485],
       ...,
       [ 1.0831249 ,  1.2077019 ,  0.25394261, ...,  0.        ,
         0.40601667,  0.520491  ],
       [ 0.67119551,  0.84381836,  0.2211079 , ...,  0.        ,
         1.03888734,  0.95640534],
       [-0.13893229,  1.41183168,  0.31961201, ...,  0.        ,
         0.40601667,  0.520491  ]])

In [101]:
#predict the profit:
data_new.loc[:, 'predicted_profit'] = clf.predict(X_new)

/var/folders/wx/yg6t278534bcmkcnbt_lz9zw0000gn/T/ipykernel_64079/1053935968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new.loc[:, 'predicted_profit'] = clf.predict(X_new)


In [103]:
data_new.groupby('location_id')['predicted_profit'].sum()

location_id
11    381407.403956
12    384135.842769
13    370910.810672
Name: predicted_profit, dtype: float64